In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
import gzip
import numpy as np

Let's start by getting familiar with the data we'll be using, the Fashion MNIST dataset. This dataset contains 70,000 grayscale images of articles of clothing &mdash; 60,000 meant to be used for training and 10,000 meant for testing. The images are square and contain 28 &times; 28 = 784 pixels, where each pixel is represented by a value between 0 and 255. Each of these images is associated with a label, which is an integer between 0 and 9 that classifies the article of clothing. The following dictionary helps us understand the clothing categories corresponding to these integer labels:

让我们首先熟悉我们将使用的数据，即 Fashion MNIST 数据集。 该数据集包含 70,000 张服装的灰度图像—— 60,000 个用于训练，10,000 个用于测试。 图像是正方形的，包含 28 × 28 = 784 个像素，其中每个像素由 0 到 255 之间的值表示。这些图像中的每一个都与一个标签相关联，标签是一个 0 到 9 之间的整数，用于对服装进行分类。 下面的字典帮助我们理解这些整数标签对应的服装类别：

In [2]:
labels_map = {
  0: 'T-Shirt',
  1: 'Trouser',
  2: 'Pullover',
  3: 'Dress',
  4: 'Coat',
  5: 'Sandal',
  6: 'Shirt',
  7: 'Sneaker',
  8: 'Bag',
  9: 'Ankle Boot',
}

If you're following this code but executing it outside of this Microsoft Learn sandbox notebook environment, you can load Fashion MNIST by calling the [load_data()](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/fashion_mnist/load_data) method of the Fashion MNIST dataset in the Keras API, as you can see in the commented out code below. Keras provides an easy way to get not only Fashion MNIST, but many other popular datasets through [tk.keras.datasets](https://www.tensorflow.org/api_docs/python/tf/keras/datasets). In this Learn sandbox notebook we already have the data locally, so we'll load it directly instead, and create four NumPy arrays containing the training and test data and labels. 

如果您遵循此代码但在此 Microsoft Learn 沙盒笔记本环境之外执行它，则可以通过调用 [load_data()](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/fashion_mnist/load_data) Keras API 中 Fashion MNIST 数据集的方法，如您在下面注释掉的代码中所见。 Keras 提供了一种简单的方法，不仅可以通过 [tk.keras.datasets](https://www.tensorflow.org/api_docs/python/tf/keras/datasets) 获取 Fashion MNIST，还可以获取许多其他流行的数据集。 在此 Learn 沙盒笔记本中，我们已在本地拥有数据，因此我们将直接加载它，并创建四个包含训练和测试数据及标签的 NumPy 数组。

In [3]:
(training_images, training_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

def read_images(path: str, image_size: int, num_items: int) -> np.ndarray:
  with gzip.open(path, 'rb') as file:
    data = np.frombuffer(file.read(), np.uint8, offset=16)
    data = data.reshape(num_items, image_size, image_size)
  return data

def read_labels(path: str, num_items: int) -> np.ndarray:
  with gzip.open(path, 'rb') as file:
    data = np.frombuffer(file.read(num_items + 8), np.uint8, offset=8)
    data = data.astype(np.int64)
  return data

image_size = 28
num_train = 60000
num_test = 10000

training_images = read_images('data/FashionMNIST/raw/train-images-idx3-ubyte.gz', image_size, num_train)
test_images = read_images('data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz', image_size, num_test)
training_labels = read_labels('data/FashionMNIST/raw/train-labels-idx1-ubyte.gz', num_train)
test_labels = read_labels('data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz', num_test)

Now that we have the data, we can display a sampling of images and corresponding labels from the training data, to get a feel for the data we'll be working with.

现在我们有了数据，我们可以显示训练数据中的图像样本和相应标签，以了解我们将要处理的数据。

In [ ]:
import random
import matplotlib.pyplot as plt

figure = plt.figure(figsize=(8, 8))
cols = 3
rows = 3
for i in range(1, cols * rows + 1):
  sample_idx = random.randint(0, len(training_images))
  image = training_images[sample_idx]
  label = training_labels[sample_idx]
  figure.add_subplot(rows, cols, i)
  plt.title(labels_map[label])
  plt.axis('off')
  plt.imshow(image.squeeze(), cmap='gray')
plt.show()


Let's inspect the first label in the training data:

让我们检查训练数据中的第一个标签：

In [ ]:
training_labels[0]

As we saw in the `labels_map` dictionary we printed earlier, this corresponds to an "Ankle boot." Now let's inspect the image that corresponds to this label: 

正如我们在之前打印的 `labels_map` 字典中看到的，这对应于“Ankle boot”。 现在让我们检查对应于此标签的图像： 

In [ ]:
training_images[0]

An image is composed of pixel values, and each pixel in the image is of type unsigned int and contains a value between 0 and 255.

For such a small dataset, we could just use the NumPy arrays given to us by Keras to train the neural network. However, if we had a large dataset, we would need to wrap it in a [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) instance, which handles large data better by making it easy to keep just a portion of it in memory. We've decided to wrap our data in a `Dataset` in this sample, so you're prepared to work with large data in the future.

图像由像素值组成，图像中的每个像素都是unsigned int类型，包含0到255之间的值。

对于这么小的数据集，我们可以只使用 Keras 提供给我们的 NumPy 数组来训练神经网络。 然而，如果我们有一个大的数据集，我们就需要把它包装在一个[tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)实例中，这样可以更好地处理大数据，使其在内存中只保留一部分数据。 我们决定在此示例中将我们的数据包装在`Dataset`中，以便您为将来处理大数据做好准备。

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((training_images, training_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

You saw earlier that each pixel of the image is represented by an unsigned int. In machine learning, we generally want the pixel values of our training data to be floating-point numbers between 0 and 1, so we convert them in the following way:

您之前看到图像的每个像素都由一个 unsigned int 表示。 在机器学习中，我们一般希望我们的训练数据的像素值是0到1之间的浮点数，所以我们按如下方式进行转换：

In [ ]:
train_dataset = train_dataset.map(lambda image, label: (float(image) / 255.0, label))
test_dataset = test_dataset.map(lambda image, label: (float(image) / 255.0, label))

You may have noticed that each value returned by the `Dataset` is a tuple containing an image and a label. We divide each value in the image by 255, and we keep the label as is. Let's inspect the values of the same image we inspected earlier, to see the difference. 

您可能已经注意到，`Dataset` 返回的每个值都是一个包含图像和标签的元组。 我们将图像中的每个值除以 255，并保持标签不变。 让我们检查一下我们之前检查过的同一图像的值，看看有什么不同。

In [ ]:
train_dataset.as_numpy_iterator().next()[0]

As expected, the pixel values are now floating-point numbers between 0 and 1. 

Notice that now that we have a `Dataset`, we can no longer index it the same way as a NumPy array. Instead, we get an iterator by calling the `as_numpy_iterator` method, and we advance it by calling its `next` method. At this point, we have a tuple containing an image and the corresponding label, so we can get the element at index 0 to inspect the image.

Finally, we tell the `Dataset` to give us batches of data of size 64, and we shuffle the data:

正如预期的那样，像素值现在是 0 到 1 之间的浮点数。

请注意，现在我们有了一个`Dataset`，我们不能再像 NumPy 数组一样对它进行索引。 相反，我们通过调用 `as_numpy_iterator` 方法获得一个迭代器，并通过调用其` next` 方法推进它。 此时，我们有一个包含图像和相应标签的元组，因此我们可以获取索引 0 处的元素来检查图像。

最后，我们告诉 `Dataset` 给我们一批大小为 64 的数据，然后我们对数据进行打乱：

In [ ]:
batch_size = 64
train_dataset = train_dataset.batch(batch_size).shuffle(500)
test_dataset = test_dataset.batch(batch_size).shuffle(500)

By specifying the batch size, we're telling the `Dataset` that when we iterate over it, we want to receive not one, but a batch of 64 items instead. If we print the length of the first item returned by the iterator, we'll see that we in fact get 64. 

通过指定批量大小，我们告诉`Dataset`，当我们迭代它的时候，我们想收到的不是一个，而是一批64个项目。如果我们打印迭代器返回的第一个项目的长度，我们会看到我们实际上得到了64个。

In [ ]:
len(train_dataset.as_numpy_iterator().next()[0])

Why do we want to get batches of 64 when iterating over the `Dataset`? We'll come back to that in the section on training. But first we need to learn about the neural network architecture we'll use for this sample.

为什么我们要在迭代`Dataset`时获得 64 个批次？ 我们将在有关培训的部分中回到这一点。 但首先我们需要了解我们将用于此示例的神经网络架构。